In [62]:
import sys
import numpy as np
import pandas as pd 
import os

from glob import glob
from typing import Tuple

import matplotlib.pyplot as plt

sys.path.insert(0, '../')

from pickle import load, dump
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

from src.vg2signal import read_raw_vg_as_df, make_smoother, make_shoulder_getter, make_detilter
from src.generate_dataset import v2signal_extra_features

from src.utils  import per_error
from src.config import DATASET_PATH, models_features_per, models_features_r2

In [63]:
# Testing only on ML1 and ML2 dataset
def v2signal_raw(vg_filename: str,
             do_log: bool,
             smoothing_bw: float,
             vcenter: float,
             vwidth: float,
             stiffness: float):

    vg_df = read_raw_vg_as_df(vg_filename)

    if (vg_df['I'].to_numpy() < 0).any():
        temp = [None] * 11
        return [None, None, vg_df] + temp

    if do_log:
        cur_var_name = "logI"
        #vg_df[cur_var_name] = np.emath.logn(logbase, vg_df["I"])
        vg_df[cur_var_name] = np.log2(vg_df["I"])
    else:
        cur_var_name = "I"

    smoother = make_smoother(smoothing_bw)

    vg_df["smoothed"] = smoother(vg_df["V"], vg_df[cur_var_name].to_numpy())

    shoulder_getter = make_shoulder_getter(1, 1.1)
    (peak_signal, peak_v_shoulder) = shoulder_getter(vg_df["V"],
                                                     vg_df["smoothed"])

    vcenter = peak_v_shoulder
    vstart = vcenter - 0.5*vwidth
    vend = vcenter + 0.5*vwidth

    detilter = make_detilter(vstart, vend, stiffness)
    vg_df["detilted"] = detilter(vg_df["V"].to_numpy(),
                                 vg_df["smoothed"].to_numpy())

   
    return  vg_df

def run_vg2_raw(folderpath: str, 
            do_log:bool, 
            recenter:bool, 
            smoothing_bw:float, 
            stiffness:float, 
            vcenter:float, 
            vwidth1:float, 
            vwidth2:float) -> Tuple[dict, str]:

    os.chdir(folderpath)  # change to desired folderpath

    signal_lst = []
    
    for filename in os.listdir():
        if filename[-3:] == 'txt':
            print("Analyzing:", filename)
            df = v2signal_raw(filename,
                                        do_log,
                                        smoothing_bw,
                                        vcenter,
                                        vwidth1,
                                        stiffness)

            (peak_signal, peak_curvature, peak_v, vg_df, vcenter, ph, signal_mean, signal_std,\
                dS_dV_max_peak, dS_dV_min_peak, dS_dV_peak_diff, dS_dV_max_V, \
                dS_dV_min_V, dS_dV_area) = v2signal_extra_features(filename,
                                                                    do_log,
                                                                    smoothing_bw,
                                                                    vcenter,
                                                                    vwidth1,
                                                                    stiffness)
            
            if (peak_signal == None) or (peak_curvature==None):
                #print(f"peak_signal:{peak_signal} OR peak curvature: {peak_curvature}", filename)
                continue

            idx1 = filename.rfind("cbz")
            idx2 = filename[idx1:].find("_")
            conc = filename[idx1 + 3:idx1 + idx2]
            replicate = filename[idx1 + idx2 + 1:filename.rfind(".")]

            # Crop only the signal
            try:
                crop = (df['V']<1.15) & (df['V']>0.93)
                VI   =  df['detilted'][crop].to_numpy() 
            except:
                continue

            if 'p' in conc:  
                pi = conc.find('p')
                conctemp = conc[:pi] + '.' + conc[pi + 1:]
                conc = conctemp

            signal_lst.append([filename, round(peak_signal, 4), round(peak_curvature, 4), round(peak_v, 4), round(vcenter, 4), ph, round(signal_mean, 4), round(signal_std, 4),\
                              round(dS_dV_max_peak, 4), round(dS_dV_min_peak, 4), round(dS_dV_peak_diff, 4), round(dS_dV_max_V, 4), round(dS_dV_min_V, 4), round(dS_dV_area, 4), [VI]])  # add text filename & peak signal to signal list
            
            concstrxl   = str(float(conc))

    signal_df = pd.DataFrame(signal_lst, columns=["file", "peak area", "peak curvature", "peak V", "vcenter", "PH", "signal_mean", "signal_std", \
                              "dS_dV_max_peak", "dS_dV_min_peak", "dS_dV_peak_diff", "dS_dV_max_V", "dS_dV_min_V", "dS_dV_area", 'raw'])

    
    with open(f"raw_data_feature_extracted.pickle", 'wb') as f:
        dump(signal_df, f)
   
    return    signal_df  

def load_dataset(dataset_path=None, raw=False, normalize=True) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    if dataset_path==None: dataset_path = DATASET_PATH

    if ('ML1_ML2'in os.path.basename(dataset_path)) or ('test' in os.path.basename(dataset_path)):
        datasets = sorted([f"{i}/raw_data_feature_extracted.pickle" for i in glob(f'{dataset_path}/*')])
        
        df = []
        for dataset in datasets:

            with open(dataset, 'rb') as f:
                data = load(f)
                df.append(data) 
        df = pd.concat(df)

    else:
        dataset_path = f"{dataset_path}/raw_data_feature_extracted.pickle"
        with open(dataset_path, 'rb') as f:
            df = load(f)

    if raw: X = [x.iloc[0] for (_,x) in df[['raw']].iterrows()] 
    else:   
         
        X = df.drop(columns=['raw', 'file'], inplace=False)
        X.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \
                        'dS_dV_area':'univariate, area(dS/dV)', 'dS_dV_max_peak':'univariate, max(dS/dV)', 'dS_dV_min_peak':'univariate, min(dS/dV)',\
                    'dS_dV_peak_diff':'univariate, max(dS/dV) - min(dS/dV)', \
                    'peak V':'univariate, V_max(S)', 'dS_dV_max_V':'univariate, V_max(dS/dV)', 'dS_dV_min_V':'univariate, V_min(dS/dV)',\
        }, inplace = True)
        columns = X.columns
  
    X = np.array(X).squeeze()
    y = df['file'].apply(lambda x: int(x.split('_')[-2].replace('cbz','')))

    # Split the total dataset into training (60%) and testing (40%) dataset
    X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=20, stratify=y)


    if raw:
        X_train = pd.DataFrame(X_train)
        X_test  = pd.DataFrame(X_test)
        y_train = pd.Series(y_train)
        y_test  = pd.Series(y_test)

    else:
        scalar = StandardScaler()
        scalar.fit(X_train)

        if normalize:
            X_train = scalar.transform(X_train)
            X_test  = scalar.transform(X_test)
    
        X_train = pd.DataFrame(X_train, columns=columns)
        X_test  = pd.DataFrame(X_test, columns=columns)
        y_train = pd.Series(y_train)
        y_test  = pd.Series(y_test)

   
    
    return (X_train, X_test, y_train, y_test), (X, y)

In [64]:
(X_train, X_test, y_train, y_test), (X, y) = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1', raw=False)

In [65]:
X_train

,"univariate, area(S)",peak curvature,"univariate, V_max(S)",vcenter,"univariate, max(S)","univariate, mean(S)","univariate, std(S)","univariate, max(dS/dV)","univariate, min(dS/dV)","univariate, max(dS/dV) - min(dS/dV)","univariate, V_max(dS/dV)","univariate, V_min(dS/dV)","univariate, area(dS/dV)"
0,-1.132022,-0.941434,-2.741588,-1.141730,-1.109992,-1.130688,-1.157463,-1.080679,1.111118,-1.098497,-0.312363,-2.941396,-1.110340
1,-0.765569,-0.276131,1.279647,1.631043,-0.746128,-0.768284,-0.781584,-0.815667,0.476454,-0.635598,1.129313,1.555005,-0.745678
2,-0.008897,-0.118773,0.316458,0.163104,-0.067640,-0.017589,-0.060302,-0.160256,0.126023,-0.142141,0.264307,0.318495,-0.067238
3,-1.056215,-1.027148,-0.217588,-1.794147,-1.048638,-1.053030,-1.076192,-1.093993,0.939546,-1.013100,-1.898207,-0.580785,-1.048553
4,0.558365,0.401199,0.488115,0.652417,0.559295,0.551903,0.549232,0.482073,-0.492749,0.488417,0.264307,0.318495,0.559107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.341454,0.185777,0.332352,0.489313,0.319940,0.344815,0.305418,0.218797,-0.133713,0.173652,0.408475,0.318495,0.320442
74,-0.634201,-0.784757,0.984014,0.163104,-0.627220,-0.638854,-0.649518,-0.597080,0.565125,-0.580865,0.985145,0.880545,-0.626951
75,0.038779,-0.028346,0.453148,0.489313,-0.004823,0.034183,-0.009508,-0.067287,0.180177,-0.127701,0.696810,0.430905,-0.004240
76,0.916343,0.880226,0.621627,0.652417,0.818545,0.914307,0.864157,0.778113,-0.779005,0.779710,0.408475,0.655725,0.818368


In [ ]:
all_dataset = glob('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/*')
    
do_log   = True
recenter = False
bw       = 0.006
s        = 0
c        = 1.04
w1       = 0.15
w2       = 0.17

print(all_dataset)
for dataset_path in all_dataset:
    print(dataset_path)
    data = run_vg2_raw(dataset_path, do_log, recenter, bw, s, c, w1, w2)

In [85]:
def combine_all_data(add_all_dataset=False, raw=True):
    if add_all_dataset:
        (ML1_X_train, ML1_X_test, ML1_y_train, ML1_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1', raw=raw)
        (ML2_X_train, ML2_X_test, ML2_y_train, ML2_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_22_ML2', raw=raw)
        (ML4_X_train, ML4_X_test, ML4_y_train, ML4_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML4/')
    
        X_train = pd.concat([ML1_X_train, ML2_X_train, ML4_X_train], axis=0)
        X_test  = pd.concat([ML1_X_test,  ML2_X_test,  ML4_X_test],  axis=0)
        y_train = pd.concat([ML1_y_train, ML2_y_train, ML4_y_train], axis=0)
        y_test  = pd.concat([ML1_y_test,  ML2_y_test,  ML4_y_test],  axis=0)
        
    else:
        (X_train, X_test, y_train, y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2', raw=raw)
        y_train, y_test = y_train.to_numpy(), y_test.to_numpy()

    return X_train, X_test, y_train, y_test
    

In [86]:
X_train, X_test, y_train, y_test = combine_all_data(add_all_dataset=False, raw=False)

In [87]:
X_train.head()

,"univariate, area(S)",peak curvature,"univariate, V_max(S)",vcenter,"univariate, max(S)","univariate, mean(S)","univariate, std(S)","univariate, max(dS/dV)","univariate, min(dS/dV)","univariate, max(dS/dV) - min(dS/dV)","univariate, V_max(dS/dV)","univariate, V_min(dS/dV)","univariate, area(dS/dV)"
0,-1.124885,-0.873945,0.087811,-1.827358,-1.129981,-1.116253,-1.145196,-1.145603,1.010617,-1.075979,-1.471324,-0.441426,-1.129649
1,1.238794,1.283919,0.635785,0.892861,1.307136,1.248902,1.294273,1.344663,-1.378694,1.366201,0.710614,0.502914,1.307415
2,-1.159712,-0.503582,-1.414581,-1.827358,-1.150254,-1.168812,-1.186024,-1.123016,1.164540,-1.148032,-2.289551,-0.846144,-1.150323
3,1.152572,1.216019,0.225712,0.212806,1.143772,1.143784,1.151376,1.077314,-1.091191,1.087391,0.437872,0.233102,1.143242
4,1.132472,1.291898,0.283775,0.552834,1.270235,1.143784,1.212617,1.558197,-1.351637,1.451257,0.301501,0.098197,1.269716


## Feature Dataset

In [88]:
kernel  = 1**2 * RBF(length_scale=2.5)
y_LOD   = 0.9117010154341669

# R2 Score
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=2)
gp_model.fit(X_train[models_features_r2['GP']], y_train)

y_pred   = gp_model.predict(X_test[models_features_r2['GP']])
score_r2 = r2_score(y_test, y_pred)

# % error
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=2)
gp_model.fit(X_train[models_features_per['GP']], y_train)

y_pred   = gp_model.predict(X_test[models_features_per['GP']])
diff_err = per_error(y_test, np.abs(y_pred), y_LOD)

print("R2 Score", score_r2, "% error", diff_err)

R2 Score 0.8467296888609956 % error 26.42969145788004


## Modeling Raw Signal

In [98]:
X_train, X_test, y_train, y_test = combine_all_data(add_all_dataset=False, raw=True)

In [99]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0.000272,0.000083,-0.000050,0.000026,0.000237,0.000485,0.000703,0.001075,0.001827,0.002800,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.008383,0.005254,0.003036,0.001576,0.000675,0.000205,0.000036,0.000000,0.000000,0.0
2,-0.000821,-0.000784,-0.000203,0.000775,0.001672,0.002137,0.002425,0.002816,0.003212,0.003432,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003044,0.001450,0.000432,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003453,0.002297,0.001543,0.000900,0.000334,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,0.000000,0.000000,0.000179,0.000832,0.002086,0.003575,0.004696,0.005229,0.005517,0.006096,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.009836,0.005552,0.002641,0.001010,0.000265,0.000000,0.000000,0.000000,0.000000,0.0
144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002417,0.001381,0.000758,0.000392,0.000134,0.000000,0.000000,0.000000,0.000000,0.0
145,-0.000061,-0.000264,-0.000424,-0.000430,-0.000211,0.000323,0.001166,0.002097,0.002807,0.003209,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [95]:
kernel   = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1)

In [96]:
gp_model.fit(X_train, y_train)

GaussianProcessRegressor(alpha=1, kernel=1**2 * RBF(length_scale=1))

In [97]:
y_pred = gp_model.predict(X_test)

score_r2 = r2_score(y_test, y_pred)
diff_err = per_error(y_test, np.abs(y_pred), y_LOD)

print("R2 Score", score_r2, "% error", diff_err)

R2 Score 0.8357769354179536 % error 25.605102515550847
